# Weather Data #

In [39]:
# imports
import pandas as pd
import numpy as np
import time

In [40]:
# define data location
location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
#location = '/Users/loki/Documents/Data/Forecasting Project/'

## Data Cleaning & Checking 

In [41]:
# weather data
temp_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="TMP")
humid_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="HUM")
wind_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="WSP")
cloud_in = pd.read_excel(location+'Weather Data for Drexel 9_28_2018.xlsx', sheet_name="CC")

In [42]:
# clean for hourly
hr_temp = temp_in.drop(columns=['Avg','HighDB','LowDB','AvgHL','Gas Day Average','HDD-HL','CDD-HL','HDD-24','CDD-24',
                            'Unnamed: 34','Unnamed: 35'])
hr_humid = humid_in.drop(columns=['Avg','Unnamed: 26','Unnamed: 27','Unnamed: 28'])
hr_wind = wind_in.drop(columns=['Avg','Unnamed: 26'])
hr_cloud = cloud_in.drop(columns=['AvgDaytime','Avg'])

# convert to datetime
hr_temp['Dt'] =  pd.to_datetime(hr_temp['Dt'])
hr_humid['Dt'] =  pd.to_datetime(hr_humid['Dt'])
hr_wind['Dt'] =  pd.to_datetime(hr_wind['Dt'])
hr_cloud['Dt'] =  pd.to_datetime(hr_cloud['Dt'])

In [43]:
# clean for daily
day_temp = temp_in.drop(columns=['HR0','HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8','HR9','HR10','HR11','HR12',
                             'HR13','HR14','HR15','HR16','HR17','HR18','HR19','HR20','HR21','HR22','HR23',
                             'AvgHL','Gas Day Average','HDD-HL','CDD-HL','HDD-24','CDD-24',
                             'Unnamed: 34','Unnamed: 35'])
day_temp = day_temp.rename(columns={'Avg':'AvgTemp'})

day_humid = humid_in.drop(columns=['HR0','HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8','HR9','HR10','HR11','HR12',
                               'HR13','HR14','HR15','HR16','HR17','HR18','HR19','HR20','HR21','HR22','HR23',
                               'Unnamed: 26','Unnamed: 27','Unnamed: 28'])
day_humid = day_humid.rename(columns={'Avg':'AvgHumid'})

day_wind = wind_in.drop(columns=['HR0','HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8','HR9','HR10','HR11','HR12',
                             'HR13','HR14','HR15','HR16','HR17','HR18','HR19','HR20','HR21','HR22','HR23',
                             'Unnamed: 26'])
day_wind = day_wind.rename(columns={'Avg':'AvgWind'})

day_cloud = cloud_in.drop(columns=['HR0','HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8','HR9','HR10','HR11','HR12',
                               'HR13','HR14','HR15','HR16','HR17','HR18','HR19','HR20','HR21','HR22','HR23'])
day_cloud = day_cloud.rename(columns={'AvgDaytime':'AvgDaytimeCloud', 'Avg':'AvgCloud'})

# convert to datetime
day_temp['Dt'] =  pd.to_datetime(day_temp['Dt'])
day_humid['Dt'] =  pd.to_datetime(day_humid['Dt'])
day_wind['Dt'] =  pd.to_datetime(day_wind['Dt'])
day_cloud['Dt'] =  pd.to_datetime(day_cloud['Dt'])

In [44]:
def lagDeltas(data, colname, newname, lag):
    """Calculates specified lag of provided column and saves as a new column
     
    Parameters
    ----------
    data : pandas dataframe object to be used calculate lag 0 and lag 1 deltas
    colname : name of column to be lagged
    newname : name of result to be added to pandas dataframe
    lag : number of lags to take 

    Returns
    ----------
    lagData : pandas dataframe with new column
    """
    
    data[newname] = data[colname].shift(lag)
    data[newname+'delta'] = data[colname]-data[newname]
    
    return data

## Daily Data Restructuring##

In [45]:
# sort by date
day_temp = day_temp.sort_values(by=["Dt"])
day_humid = day_humid.sort_values(by=["Dt"])
day_wind = day_wind.sort_values(by=["Dt"])
day_cloud = day_cloud.sort_values(by=["Dt"])

In [46]:
# Find lags of weather variables
# Lag temp
temp = lagDeltas(day_temp, "AvgTemp", "Temp1", 1)
temp = lagDeltas(temp, "HighDB", "HighDB1", 1)
temp = lagDeltas(temp, "LowDB", "LowDB1", 1)
humid = lagDeltas(day_humid, "AvgHumid", "Humid1", 1)
wind = lagDeltas(day_wind, "AvgWind", "Wind1", 1)
cloud = lagDeltas(day_cloud, "AvgCloud", "Cloud1", 1)

In [47]:
# Merge
weather1 = pd.merge(temp, humid, how='inner', on=['Dt'])
weather2 = pd.merge(wind, cloud, how='inner', on=['Dt'])
daily_weather = pd.merge(weather1, weather2, how='inner', on=['Dt'])

In [48]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
daily_weather.to_pickle(location+'daily_weather.pkl.zip')

## Hourly Data Restructuring ##

In [49]:
# Tidy / Stack data (transform into tall data - one row per customer per hour):
# ref: http://www.jeannicholashould.com/tidy-data-in-python.html
tidy_temp = pd.melt(hr_temp, 
                    id_vars=['Dt'],
                    var_name='Hour', value_name='Temp')

tidy_humid = pd.melt(hr_humid,
                     id_vars=['Dt'],
                     var_name='Hour', value_name='Humid')

tidy_wind = pd.melt(hr_wind, 
                    id_vars=['Dt'],
                    var_name='Hour', value_name='Wind')

tidy_cloud = pd.melt(hr_cloud, 
                     id_vars=['Dt'],
                     var_name='Hour', value_name='Cloud')

# relabel & retype for sorting
tidy_temp['Hour'] = tidy_temp['Hour'].str.extract('(\d+)').astype(int)
tidy_humid['Hour'] = tidy_humid['Hour'].str.extract('(\d+)').astype(int)
tidy_wind['Hour'] = tidy_wind['Hour'].str.extract('(\d+)').astype(int)
tidy_cloud['Hour'] = tidy_cloud['Hour'].str.extract('(\d+)').astype(int)

In [50]:
# sort by date & time
tidy_temp = tidy_temp.sort_values(by=["Dt","Hour"])
tidy_humid = tidy_humid.sort_values(by=["Dt","Hour"])
tidy_wind = tidy_wind.sort_values(by=["Dt","Hour"])
tidy_cloud = tidy_cloud.sort_values(by=["Dt","Hour"])

In [51]:
# Find lags of weather variables
# Lag temp
for i in range(1,7):
    temp = lagDeltas(tidy_temp, "Temp", "Temp"+str(i), i)

# Lag humid
for i in range(1,7):
    humid = lagDeltas(tidy_humid, "Humid", "Humid"+str(i), i)
    
# Lag wind
for i in range(1,7):
    wind = lagDeltas(tidy_wind, "Wind", "Wind"+str(i), i)

# Lag cloud
for i in range(1,7):
    cloud = lagDeltas(tidy_cloud, "Cloud", "Cloud"+str(i), i)


In [52]:
# Merge
weather1 = pd.merge(temp, humid, how='inner', on=['Dt','Hour'])
weather2 = pd.merge(wind, cloud, how='inner', on=['Dt','Hour'])
hourly_weather = pd.merge(weather1, weather2, how='inner', on=['Dt', 'Hour'])

In [53]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
hourly_weather.to_pickle(location+'hourly_weather.pkl.zip')

In [54]:
hourly_weather.head()

,Dt,Hour,Temp,Temp1,Temp1delta,Temp2,Temp2delta,Temp3,Temp3delta,Temp4,...,Cloud2,Cloud2delta,Cloud3,Cloud3delta,Cloud4,Cloud4delta,Cloud5,Cloud5delta,Cloud6,Cloud6delta
0,2000-01-01,0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-01,1,37.0,35.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-01,2,37.0,37.0,0.0,35.0,2.0,NaN,NaN,NaN,...,100.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-01,3,35.0,37.0,-2.0,37.0,-2.0,35.0,0.0,NaN,...,100.0,0.0,100.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-01,4,33.0,35.0,-2.0,37.0,-4.0,37.0,-4.0,35.0,...,100.0,0.0,100.0,0.0,100.0,0.0,NaN,NaN,NaN,NaN


## Merge with Use ##

In [55]:
# to combine with use data:
daily_weather = pd.merge(daily_tall, daily_weather, how='inner', on=['Dt'])
hourly_weather = pd.merge(hourly_tall, hourly_weather, how='inner', on=['Dt','Hour'])
# CustIDs || Date | Consumption |||| Weather_variables 

NameError: name 'daily_tall' is not defined